In [ ]:
import pickle

import matplotlib.pyplot as plt
import numpy as np

from laboneq import __version__ as laboneq_version
from laboneq_benchmark import benchmark, extract_duration, extract_metadata
from laboneq_benchmark.device_setups import (
    create_benchmark_device_setup,
    create_benchmark_device_setup_qpu,
)
from laboneq_benchmark.experiments import (
    ramsey_parallel,
    rb_parallel,
    # amplitude_rabi_parallel,
    two_qubit_RB,
)

## Quickstart

In [ ]:
qubit_counts = [1, 2]

benchmark_results = []
for n_qubit in qubit_counts:
    device_setup = create_benchmark_device_setup(
        number_of_qubits=n_qubit, calibrate_setup=False
    )
    experiment = rb_parallel(
        qubits=list(device_setup.qubits.values())[:n_qubit],
        sequence_exponent=8,
        chunk_count=10,
        n_sequences_per_length=20,
        do_readout=False,
        prng_seed=42,
    )
    benchmark_result = benchmark(
        device_setup=device_setup,
        experiment=experiment,
        metadata={"qubits.count": n_qubit, "number_average": 2**12},
    )
    benchmark_results.append(benchmark_result)

## API overview

To dictionary format, e.g. for saving

In [ ]:
benchmark_results[0].to_dict()

Operations query

In [ ]:
# Available operations
benchmark_results[0].operation_names()

In [ ]:
# Query specific operation
benchmark_results[0].get_operation("experiment-compile").to_dict()

Benchmark run metadata

In [ ]:
benchmark_results[0].metadata

## Benchmark runs - parallel Ramsey

Create and compile a Ramsey experiment in parallel on a configurable number of qubits. 

Tested to run with LabOne Q versions from 2.41.0.

In [ ]:
number_of_qubits = range(12, 193, 24)
number_sweep_steps = 201

benchmark_results_ramsey = []
for number in number_of_qubits:
    print(f"Generating and compiling parallel Ramsey experiment for {number} qubits")
    device_setup = create_benchmark_device_setup(
        number_of_qubits=number, calibrate_setup=False
    )
    experiment = ramsey_parallel(
        qubits=list(device_setup.qubits.values())[:number],
        number_sweep_steps=number_sweep_steps,
        chunk_count=1,
    )
    benchmark_result = benchmark(
        device_setup=device_setup,
        experiment=experiment,
        metadata={
            "number_sweep_steps": number_sweep_steps,
            "qubits.count": number,
        },
    )
    benchmark_results_ramsey.append(benchmark_result)

In [ ]:
with open(f"data/Ramsey/benchmark_results_ramsey_{laboneq_version}.pkl", "wb") as f:
    pickle.dump(benchmark_results_ramsey, f)

In [ ]:
qubit_numbers_ramsey = extract_metadata(benchmark_results_ramsey, "qubits.count")
compilation_duration_ramsey = extract_duration(
    benchmark_results_ramsey, "experiment-compile"
)
code_generator_duration_ramsey = extract_duration(
    benchmark_results_ramsey, "laboneq.compiler.generate-code"
)

plot data

In [ ]:
plt.plot(
    qubit_numbers_ramsey,
    compilation_duration_ramsey,
    label=f"Total Compilation time - version {laboneq_version}",
)
plt.plot(
    qubit_numbers_ramsey,
    code_generator_duration_ramsey,
    label=f"Code Generation time - version {laboneq_version}",
)
plt.xlabel("Number of qubits in Ramsey experiment")
plt.ylabel("Code generation & Compilation time [s]")
plt.legend()
plt.show()

## Benchmark runs - parallel single qubit RB

Create and compile a randomized benchmarking experiment in parallel on a configurable number of qubits for a specific clifford sequence length. 

Tested to run with LabOne Q versions from 2.41.0.

In [ ]:
sequence_exponents = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
number_of_qubits = 4

benchmark_results_rb = []
for sequence_exponent in sequence_exponents:
    print(
        f"Generating and compiling parallel RB experiment for {number_of_qubits} qubits and {2**sequence_exponent} clifford gates"
    )
    device_setup = create_benchmark_device_setup(
        number_of_qubits=number_of_qubits, calibrate_setup=False
    )
    experiment = rb_parallel(
        qubits=list(device_setup.qubits.values())[:number_of_qubits],
        sequence_exponent=sequence_exponent,
        chunk_count=20,
        n_sequences_per_length=40,
    )
    benchmark_result = benchmark(
        device_setup=device_setup,
        experiment=experiment,
        metadata={
            "qubits.count": number_of_qubits,
            "sequence_exponent": sequence_exponent,
        },
    )
    benchmark_results_rb.append(benchmark_result)

In [ ]:
with open(f"data/RB/benchmark_results_rb_{laboneq_version}.pkl", "wb") as f:
    pickle.dump(benchmark_results_rb, f)

In [ ]:
sequence_exponents_rb = extract_metadata(benchmark_results_rb, "sequence_exponent")
compilation_duration_rb = extract_duration(benchmark_results_rb, "experiment-compile")
code_generator_duration_rb = extract_duration(
    benchmark_results_rb, "laboneq.compiler.generate-code"
)

plot data

In [ ]:
plt.plot(
    2**sequence_exponents_rb,
    compilation_duration_rb,
    label=f"Total Compilation time - version {laboneq_version}",
)
plt.plot(
    2**sequence_exponents_rb,
    code_generator_duration_rb,
    label=f"Code Generation time - version {laboneq_version}",
)
plt.xlabel("Length of RB sequence")
plt.ylabel("Code generation & Compilation time [s]")
plt.legend()
plt.show()

## Benchmark runs - two qubit RB from Qiskit

This experiment requires some functionality that is only available in later versions of LabOne Q - the earliest tested version is 2.57.0

In [ ]:
sequence_exponents = [2, 3, 4, 5, 6, 7, 8]

benchmark_results_2qb_rb = []
for sequence_exponent in sequence_exponents:
    print(
        f"Generating and compiling two-Qubit RB experiment from Qiskit with {2**sequence_exponent} clifford gates"
    )
    device_setup, qpu = create_benchmark_device_setup_qpu(
        number_of_qubits=2, calibrate_setup=False
    )
    experiment = two_qubit_RB(
        device_setup=device_setup,
        qpu=qpu,
        sequence_exponent=sequence_exponent,
        chunk_count=40,
        n_sequences_per_length=40,
    )
    benchmark_result = benchmark(
        device_setup=device_setup,
        experiment=experiment,
        metadata={
            "sequence_exponent": sequence_exponent,
        },
    )
    benchmark_results_2qb_rb.append(benchmark_result)

In [ ]:
with open(f"data/2QB_RB/benchmark_results_2qb_rb_{laboneq_version}.pkl", "wb") as f:
    pickle.dump(benchmark_results_2qb_rb, f)

In [ ]:
sequence_exponents_2qb_rb = extract_metadata(
    benchmark_results_2qb_rb, "sequence_exponent"
)
compilation_duration_2qb_rb = extract_duration(
    benchmark_results_2qb_rb, "experiment-compile"
)
code_generator_duration_2qb_rb = extract_duration(
    benchmark_results_2qb_rb, "laboneq.compiler.generate-code"
)
creation_duration_2qb_rb = extract_duration(
    benchmark_results_2qb_rb, "experiment-creation"
)

In [ ]:
plt.plot(
    2**sequence_exponents_2qb_rb,
    compilation_duration_2qb_rb,
    label=f"Total Compilation time - version {laboneq_version}",
)
plt.plot(
    2**sequence_exponents_2qb_rb,
    code_generator_duration_2qb_rb,
    label=f"Code generation - version {laboneq_version}",
)
plt.plot(
    2**sequence_exponents_2qb_rb,
    creation_duration_2qb_rb,
    label=f"Experiment creation time - version {laboneq_version}",
)
plt.xlabel("Length of two qubit RB sequence")
plt.ylabel("Experiment creation & Compilation times [s]")
plt.legend()
plt.show()

# Load and display data

In [ ]:
import glob
import re

In [ ]:
versions_to_display = [
    "2.41.0",
    "2.47.0",
    "2.57.0",
    # "2.59.0",
    "2.62.0",
    "25.10.2",
    "26.1.0b1",
]

### Ramsey benchmarking

In [ ]:
ramsey_files = glob.glob("data/Ramsey/*.pkl")
ramsey_files.sort()
ramsey_files

In [ ]:
ramsey_data = {}
for file in ramsey_files:
    version_match = re.search(r"(?<=ramsey_)(.*?)(?=.pkl)", file)
    if version_match:
        file_version = version_match.group(0)
        with open(file, "rb") as f:
            loaded_data = pickle.load(f)
        ramsey_data[file_version] = {
            "qubit_count": extract_metadata(loaded_data, "qubits.count"),
            "experiment_creation": extract_duration(loaded_data, "experiment-creation"),
            "compilation_duration": extract_duration(loaded_data, "experiment-compile"),
            "scheduler_duration": extract_duration(
                loaded_data, "laboneq.compiler.schedule"
            ),
            "code_generator_duration": extract_duration(
                loaded_data, "laboneq.compiler.generate-code"
            ),
        }
    else:
        print(f"Could not determine LabOne Q version of data in file {file}")

In [ ]:
for version in ramsey_data.keys():
    if version in versions_to_display:
        plt.loglog(
            ramsey_data[version]["qubit_count"],
            ramsey_data[version]["compilation_duration"],
            label=f"{version}",
        )
plt.title("Total Compilation Time - parallel Ramsey Experiment")
plt.xlabel("Number of qubits in Ramsey experiment")
plt.ylabel("Compilation time [s]")
plt.legend()
plt.show()

In [ ]:
for version in ramsey_data.keys():
    if version in versions_to_display:
        plt.loglog(
            ramsey_data[version]["qubit_count"],
            ramsey_data[version]["code_generator_duration"],
            label=f"{version}",
        )
plt.loglog(
    np.linspace(12, 180, 12),
    0.01 * np.linspace(12, 180, 12) ** 2,
    "--",
    label="quadratic",
)
plt.loglog(
    np.linspace(12, 180, 12), 0.005 * np.linspace(12, 180, 12), "--", label="linear"
)
plt.title("Code Generation Time - parallel Ramsey Experiment")
plt.xlabel("Number of qubits in Ramsey experiment")
plt.ylabel("Code Generation time [s]")
plt.legend()
# plt.savefig("CodeGeneration_Ramsey.svg", format="svg")
plt.show()

In [ ]:
for version in ramsey_data.keys():
    if version in versions_to_display:
        plt.plot(
            ramsey_data[version]["qubit_count"],
            100
            * ramsey_data[version]["code_generator_duration"]
            / ramsey_data[version]["compilation_duration"],
            label=f"{version}",
        )
plt.title("Fractional Code Generation Time - parallel Ramsey Experiment")
plt.xlabel("Number of qubits in Ramsey experiment")
plt.ylabel("Fractional Code Generation time [%]")
plt.legend()
# plt.savefig("CodeGeneration_Ramsey.svg", format="svg")
plt.show()

### RB benchmarking

In [ ]:
rb_files = glob.glob("data/RB/*.pkl")
rb_files.sort()
rb_files

In [ ]:
rb_data = {}
for file in rb_files:
    version_match = re.search(r"(?<=rb_)(.*?)(?=.pkl)", file)
    if version_match:
        file_version = version_match.group(0)
        # print(file_version)
        with open(file, "rb") as f:
            loaded_data = pickle.load(f)
        # print(loaded_data)
        rb_data[file_version] = {
            "sequence_exponent": extract_metadata(loaded_data, "sequence_exponent"),
            "experiment_creation": extract_duration(loaded_data, "experiment-creation"),
            "compilation_duration": extract_duration(loaded_data, "experiment-compile"),
            "scheduler_duration": extract_duration(
                loaded_data, "laboneq.compiler.schedule"
            ),
            "code_generator_duration": extract_duration(
                loaded_data, "laboneq.compiler.generate-code"
            ),
        }
    else:
        print(f"Could not determine LabOne Q version of data in file {file}")

In [ ]:
for version in rb_data.keys():
    if version in versions_to_display:
        plt.plot(
            2 ** rb_data[version]["sequence_exponent"],
            rb_data[version]["compilation_duration"],
            label=f"{version}",
        )
plt.title("Total Compilation Time - 4 qubit RB Experiment")
plt.xlabel("Number of Clifford gates")
plt.ylabel("Compilation time [s]")
plt.legend()
plt.show()

In [ ]:
for version in rb_data.keys():
    if version in versions_to_display:
        plt.plot(
            2 ** rb_data[version]["sequence_exponent"],
            rb_data[version]["code_generator_duration"],
            label=f"{version}",
        )
plt.title("Code Generation Time - 4 qubit RB Experiment")
plt.xlabel("Number of Clifford gates")
plt.ylabel("Compilation time [s]")
plt.legend()
plt.show()

### 2 Qubit RB benchmarking

In [ ]:
rb_2qb_files = glob.glob("data/2QB_RB/*.pkl")
rb_2qb_files.sort()
rb_2qb_files

In [ ]:
rb_2qb_data = {}
for file in rb_2qb_files:
    version_match = re.search(r"(?<=rb_)(.*?)(?=.pkl)", file)
    if version_match:
        file_version = version_match.group(0)
        # print(file_version)
        with open(file, "rb") as f:
            loaded_data = pickle.load(f)
        # print(loaded_data)
        rb_2qb_data[file_version] = {
            "sequence_exponent": extract_metadata(loaded_data, "sequence_exponent"),
            "experiment_creation": extract_duration(loaded_data, "experiment-creation"),
            "compilation_duration": extract_duration(loaded_data, "experiment-compile"),
            "scheduler_duration": extract_duration(
                loaded_data, "laboneq.compiler.schedule"
            ),
            "code_generator_duration": extract_duration(
                loaded_data, "laboneq.compiler.generate-code"
            ),
        }
    else:
        print(f"Could not determine LabOne Q version of data in file {file}")

In [ ]:
for version in rb_2qb_data.keys():
    if version in versions_to_display:
        plt.plot(
            2 ** rb_2qb_data[version]["sequence_exponent"],
            rb_2qb_data[version]["compilation_duration"],
            label=f"{version}",
        )
plt.title("Total Compilation Time - 2-qubit RB Experiment from Qiskit")
plt.xlabel("Number of Clifford gates")
plt.ylabel("Compilation time [s]")
plt.legend()
plt.show()

In [ ]:
for version in rb_2qb_data.keys():
    if version in versions_to_display:
        plt.plot(
            2 ** rb_2qb_data[version]["sequence_exponent"],
            rb_2qb_data[version]["code_generator_duration"],
            label=f"{version}",
        )
plt.title("Code Generation Time - 2-qubit RB Experiment from Qiskit")
plt.xlabel("Number of Clifford gates")
plt.ylabel("Compilation time [s]")
plt.legend()
plt.show()

### Overview plots

In [ ]:
versions = []
code_times = []
for version in rb_data.keys():
    if version in versions_to_display:
        versions.append(version)
        code_times.append(rb_data[version]["code_generator_duration"][-1])

plt.plot(
    versions,
    code_times,
    label=f"RB with 8192 Cliffords - x{round(code_times[0] / code_times[-1])} improvement",
)

versions = []
code_times = []
for version in rb_2qb_data.keys():
    if version in versions_to_display:
        versions.append(version)
        code_times.append(rb_2qb_data[version]["code_generator_duration"][-1])

plt.plot(
    versions,
    code_times,
    label=f"2-Qubit RB with 256 Cliffords - x{round(code_times[0] / code_times[-1])} improvement",
)

versions = []
code_times = []
for version in ramsey_data.keys():
    if version in versions_to_display:
        versions.append(version)
        code_times.append(ramsey_data[version]["code_generator_duration"][-1])

plt.plot(
    versions,
    code_times,
    label=f"Ramsey on 180 qubits - x{round(code_times[0] / code_times[-1])} improvement",
)

plt.xticks(ticks=versions, labels=versions)
plt.xlabel("LabOne Q version")
plt.ylabel("Code Generation Duration [s]")
plt.ylim(0, 300)
plt.legend(loc="upper right")
# plt.savefig("CodeGeneration_Overview.svg", format="svg")
plt.show()

In [ ]:
versions = []
compile_times = []
for version in rb_data.keys():
    if version in versions_to_display:
        versions.append(version)
        compile_times.append(rb_data[version]["compilation_duration"][-1])

plt.plot(
    versions,
    compile_times,
    label=f"RB with 8192 gates - x{round(compile_times[0] / compile_times[-1])} improvement",
)

versions = []
compile_times = []
for version in rb_2qb_data.keys():
    if version in versions_to_display:
        versions.append(version)
        compile_times.append(rb_2qb_data[version]["compilation_duration"][-1])

plt.plot(
    versions,
    compile_times,
    label=f"2-Qubit RB with 256 Cliffords - x{round(code_times[0] / code_times[-1])} improvement",
)

versions = []
compile_times = []
for version in ramsey_data.keys():
    if version in versions_to_display:
        versions.append(version)
        compile_times.append(ramsey_data[version]["compilation_duration"][-1])

plt.plot(
    versions,
    compile_times,
    label=f"Ramsey on 180 qubits - x{round(compile_times[0] / compile_times[-1])} improvement",
)

plt.xticks(ticks=versions, labels=versions)
plt.xlabel("LabOne Q version")
plt.ylabel("Total Compilation Duration [s]")
plt.ylim(0, 340)
plt.legend(loc="upper right")
# plt.savefig("Compiler_Overview.svg", format="svg")
plt.show()

In [ ]:
versions = []
fraction_schedule = []
fraction_code = []
for version in rb_data.keys():
    if version in versions_to_display:
        versions.append(version)
        fraction_schedule.append(
            100
            * rb_data[version]["scheduler_duration"][-1]
            / rb_data[version]["compilation_duration"][-1]
        )
        fraction_code.append(
            100
            * rb_data[version]["code_generator_duration"][-1]
            / rb_data[version]["compilation_duration"][-1]
        )

plt.plot(
    versions,
    fraction_schedule,
    "--",
    label="RB - fraction of compilation time spent in scheduler",
)
plt.plot(
    versions,
    fraction_code,
    label="RB - fraction of compilation time spent in code generator",
)

versions = []
fraction_schedule = []
fraction_code = []
for version in ramsey_data.keys():
    if version in versions_to_display:
        versions.append(version)
        fraction_schedule.append(
            100
            * ramsey_data[version]["scheduler_duration"][-1]
            / ramsey_data[version]["compilation_duration"][-1]
        )
        fraction_code.append(
            100
            * ramsey_data[version]["code_generator_duration"][-1]
            / ramsey_data[version]["compilation_duration"][-1]
        )

plt.plot(
    versions,
    fraction_schedule,
    "--",
    label="Ramsey - fraction of compilation time spent in scheduler",
)
plt.plot(
    versions,
    fraction_code,
    label="Ramsey - fraction of compilation time spent in code generator",
)

plt.xticks(ticks=versions, labels=versions)
plt.xlabel("LabOne Q version")
plt.ylabel("Fraction of time [%]")
plt.ylim(0, 100)
plt.legend(loc="upper right")
# plt.savefig("Compiler_Overview.svg", format="svg")
plt.show()